In [8]:
!pip install wandb

In [1]:
import os
import numpy as np
import zipfile
import requests
from tqdm import tqdm
import torch
import torch.nn as nn
# import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
from sklearn.model_selection import train_test_split
from torch import optim
from pathlib import Path
import json
import wandb

In [2]:
dataset_url = "https://storage.googleapis.com/wandb_datasets/nature_12K.zip"
dataset_zip_path = "/kaggle/working/nature_12K.zip"
dataset_dir = "nature_12K"

In [3]:
if not os.path.exists(dataset_dir):
    if not os.path.exists(dataset_zip_path):
        print("Downloading iNaturalist-12K...")
        response = requests.get(dataset_url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        with open(dataset_zip_path, 'wb') as f, tqdm(
            desc=dataset_zip_path,
            total=total_size,
            unit='iB',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for data in response.iter_content(chunk_size=1024):
                size = f.write(data)
                bar.update(size)

/kaggle/working/nature_12K.zip: 100%|██████████| 3.55G/3.55G [00:38<00:00, 100MiB/s] 


In [4]:
if not os.path.exists(dataset_dir):
    print("Extracting dataset...")
    with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall(".")

Extracting dataset...


In [5]:
def prepare_datasets(data_dir, val_split=0.2, batch_size=32, image_size=(224, 224)):
    data_dir = Path(data_dir)

    # Define transforms (customize as needed)
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),  # Converts to [0, 1] and CxHxW
        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3),  # Normalize RGB
    ])

    # Load training and testing datasets
    full_train_dataset = ImageFolder(root=data_dir / "train", transform=transform)
    test_dataset = ImageFolder(root=data_dir / "val", transform=transform)

    # Create validation split from training set
    val_size = int(val_split * len(full_train_dataset))
    train_size = len(full_train_dataset) - val_size

    train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

    # Extract X and Y by iterating over DataLoader batches if needed
    train_loader = DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=False)
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

    # Convert to X, Y tensors
    def extract_XY(loader):
        for X, Y in loader:
            return X, Y

    X_train, Y_train = extract_XY(train_loader)
    X_val, Y_val = extract_XY(val_loader)
    X_test, Y_test = extract_XY(test_loader)

    return X_train, Y_train, X_val, Y_val, X_test, Y_test

In [6]:
# dataset_dir = "/kaggle/working/inaturalist_12K"

In [7]:
# X_train, Y_train, X_val, Y_val, X_test, Y_test = prepare_datasets(
#     data_dir='/kaggle/working/inaturalist_12K', val_split=0.2, batch_size=64, image_size=(224, 224)
# )

In [8]:
# print(X_train.shape)

# Part A
### Question 1

Build a small CNN model consisting of 5 convolution layers. Each convolution layer would be followed by an activation and a max-pooling layer.

After 5 such conv-activation-maxpool blocks, you should have one dense layer followed by the output layer containing 10 neurons. The input layer should be compatible with the images in the iNaturalist dataset dataset.
The code should be flexible such that the number of filters, size of filters, and activation function of the convolution layers and dense layers can be changed. You should also be able to change the number of neurons in the dense layer.

In [7]:
api_key = "7040d84a3ed65a967eb3389dd6fe774b418576ed" 
wandb.login(key=api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: da24m004 (da24m004-iitmaana) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [8]:
class FlexibleCNN(nn.Module):
    def __init__(self,
                 num_filters=32,  # initial number of filters
                 filter_size=3,   # size of filters (k x k)
                 activation='relu',  # activation function
                 dense_neurons=512,  # number of neurons in dense layer
                 input_channels=3,   # RGB images
                 num_classes=10,    # number of output classes
                 use_batch_norm=True,  # whether to use batch normalization
                 dropout_rate=0.2,   # dropout rate
                 filter_strategy='same'):  # filter organization strategy
        super(FlexibleCNN, self).__init__()
        
        # Store parameters for calculations
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.dense_neurons = dense_neurons
        self.use_batch_norm = use_batch_norm
        self.dropout_rate = dropout_rate
        self.filter_strategy = filter_strategy
        self.input_size = 224  # Initial image size
        
        # Choose activation function
        if activation.lower() == 'relu':
            self.activation = nn.ReLU()
        elif activation.lower() == 'leakyrelu':
            self.activation = nn.LeakyReLU()
        elif activation.lower() == 'gelu':
            self.activation = nn.GELU()
        elif activation.lower() == 'silu':
            self.activation = nn.SiLU()
        elif activation.lower() == 'mish':
            self.activation = nn.Mish()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")
        

        # Define filters based on strategy
        filter_counts = []
        if filter_strategy == 'same':
            filter_counts = [num_filters] * 5
        elif filter_strategy == 'double':
            for i in range(5):
                filter_counts.append(num_filters * (2**i))
        elif filter_strategy == 'half':
            for i in range(5):
                filter_counts.append(max(num_filters // (2**i), 8))
        elif filter_strategy == 'pyramid':
            # Increase then decrease like a pyramid
            filter_counts = [num_filters, num_filters*2, num_filters*4, num_filters*2, num_filters]
        else:
            filter_counts = [num_filters] * 5  # Default to same
            
        # Create 5 conv-activation-maxpool blocks
        self.conv_blocks = nn.ModuleList()
        in_channels = input_channels
        
        for i in range(5):
            block = []
            # Conv layer
            block.append(nn.Conv2d(in_channels, filter_counts[i], filter_size, padding=filter_size//2))

            # Batch normalization if enabled
            if use_batch_norm:
                block.append(nn.BatchNorm2d(filter_counts[i]))

            # Activation
            block.append(self.activation)

            # Max pooling
            block.append(nn.MaxPool2d(2, 2))

            # Dropout after pooling
            block.append(nn.Dropout2d(dropout_rate))

            self.conv_blocks.append(nn.Sequential(*block))
            in_channels = filter_counts[i]
            self.input_size //= 2  # Each max pool reduces size by half
        
        # Calculate the size of the flattened features after conv blocks
        # Assuming input size of 224x224 (standard for iNaturalist)
        self.final_filters = filter_counts[-1]
        self.flattened_size = self.final_filters * (self.input_size * self.input_size)
        
        # Dense layers
        self.dense = nn.Sequential(
            nn.Linear(self.flattened_size, dense_neurons),
            self.activation,
            nn.Dropout(dropout_rate),  # Dropout before final layer
            nn.Linear(dense_neurons, num_classes)
        )
    
    def forward(self, x):
        for block in self.conv_blocks:
            x = block(x)
        x = x.view(x.size(0), -1)
        x = self.dense(x)
        return x
    
    def get_computations(self):
        """Calculate total number of computations"""
        # Computations in conv layers
        # conv_computations = 0
        # input_size = 224
        # in_channels = 3
        
        # for i in range(5):
        #     # Each conv layer
        #     conv_computations += (input_size * input_size * in_channels * 
        #                         self.num_filters * self.filter_size * self.filter_size)
        #     # Each maxpool reduces size by 2
        #     input_size = input_size // 2
        #     in_channels = self.num_filters
        
        # # Computations in dense layers
        # dense_computations = (self.flattened_size * self.dense_neurons +  # first dense layer
        #                     self.dense_neurons * 10)  # output layer
        
        return sum(param.numel() for param in self.parameters()) * 100
    
    def get_parameters(self):
        """Calculate total number of parameters"""
        # Parameters in conv layers
        # conv_params = 0
        # in_channels = 3
        
        # for _ in range(5):
        #     # Each conv layer has (filter_size * filter_size * in_channels + 1) * num_filters parameters
        #     conv_params += (self.filter_size * self.filter_size * in_channels + 1) * self.num_filters
        #     # Batch norm parameters if enabled
        #     if self.use_batch_norm:
        #         conv_params += 2 * self.num_filters  # gamma and beta for each channel
        #     in_channels = self.num_filters
        
        # # Parameters in dense layers
        # dense_params = (self.flattened_size * self.dense_neurons + self.dense_neurons +  # first dense layer
        #                self.dense_neurons * 10 + 10)  # output layer
        
        return sum(p.numel() for p in self.parameters())

In [9]:
def train(config=None):
    # Initialize wandb
    with wandb.init(config=config):
        config = wandb.config
        
        # Initialize best_val_acc at the beginning of the function
        best_val_acc = 0.0

        # Set random seed for reproducibility
        torch.manual_seed(config.seed)
        np.random.seed(config.seed)

        # Data augmentation and normalization based on config
        if config.use_augmentation:
            train_transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(10),
                transforms.ColorJitter(brightness=0.2, contrast=0.2),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
            ])
        else:
            train_transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
            ])

        val_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225])
        ])
        
        # Load dataset
        train_dataset = datasets.ImageFolder(
            root='/kaggle/working/inaturalist_12K/train',  # Update to match your dataset path
            transform=train_transform
        )
        
        # Split into train and validation
        train_indices, val_indices = train_test_split(
            list(range(len(train_dataset))),
            test_size=0.2,
            stratify=train_dataset.targets,
            random_state=config.seed
        )
        
        train_subset = torch.utils.data.Subset(train_dataset, train_indices)
        val_subset = torch.utils.data.Subset(train_dataset, val_indices)
        
        train_loader = DataLoader(
            train_subset,
            batch_size=config.batch_size,
            shuffle=True,
            num_workers=4
        )
        
        val_loader = DataLoader(
            val_subset,
            batch_size=config.batch_size,
            shuffle=False,
            num_workers=4
        )
        
        # Initialize model with filter strategy
        model = FlexibleCNN(
            num_filters=config.num_filters,
            filter_size=config.filter_size,
            activation=config.activation,
            dense_neurons=config.dense_neurons,
            use_batch_norm=config.use_batch_norm,
            dropout_rate=config.dropout_rate,
            filter_strategy=config.filter_strategy
        )
        
        # Move model to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)
        
        # Loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
        
        # Update run name to include new parameters
        run_name = f"nf_{config.num_filters}_fs_{config.filter_size}_strat_{config.filter_strategy}_act_{config.activation}_dn_{config.dense_neurons}_lr_{config.learning_rate}_bs_{config.batch_size}_bn_{config.use_batch_norm}_dr_{config.dropout_rate}_aug_{config.use_augmentation}"
        
        # Set the run name
        wandb.run.name = run_name
        
        # The rest of the training code remains the same
        for epoch in range(config.epochs):
            # Training phase
            model.train()
            train_loss = 0
            train_correct = 0
            train_total = 0

            for batch_idx, (inputs, targets) in enumerate(train_loader):
                inputs, targets = inputs.to(device), targets.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                _, predicted = outputs.max(1)
                train_total += targets.size(0)
                train_correct += predicted.eq(targets).sum().item()

                if batch_idx % 100 == 0:
                    print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}')

            train_acc = 100. * train_correct / train_total

            # Validation phase
            model.eval()
            val_loss = 0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)

                    val_loss += loss.item()
                    _, predicted = outputs.max(1)
                    val_total += targets.size(0)
                    val_correct += predicted.eq(targets).sum().item()

            val_acc = 100. * val_correct / val_total

            # Log metrics to wandb
            wandb.log({
                "epoch": epoch,
                "train_loss": train_loss / len(train_loader),
                "train_acc": train_acc,
                "val_loss": val_loss / len(val_loader),
                "val_acc": val_acc
            })

            # Save best model
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(model.state_dict(), 'best_model.pth')
                print(f"Best model saved with val_acc: {val_acc:.2f}% at epoch {epoch}")

            print(f'Epoch: {epoch}, Train Loss: {train_loss/len(train_loader):.4f}, '
                  f'Train Acc: {train_acc:.2f}%, Val Loss: {val_loss/len(val_loader):.4f}, '
                  f'Val Acc: {val_acc:.2f}%')

In [10]:

# Define sweep configuration
sweep_config = {
    'method': 'bayes',  # Use Bayesian optimization
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    'parameters': {
        'num_filters': {
            'values': [16, 32, 64]
        },
        'filter_size': {
            'values': [3]
        },
        'filter_strategy': {
            'values': ['same', 'double', 'half', 'pyramid']
        },
        'activation': {
            'values': ['relu', 'gelu', 'silu', 'mish']
        },
        'dense_neurons': {
            'values': [128, 256]
        },
        'learning_rate': {
            'min': 1e-4,
            'max': 1e-2
        },
        'batch_size': {
            'values': [32, 64, 128]
        },
        'use_batch_norm': {
            'values': [True, False]
        },
        'dropout_rate': {
            'values': [0.2, 0.3, 0.4]
        },
        'use_augmentation': {
            'values': [True, False]
        },
        'epochs': {
            'value': 7
        },
        'seed': {
            'value': 42
        }
    }
}

# Initialize sweep
sweep_id = wandb.sweep(sweep_config, project="inaturalist-cnn-sweep_sample_02")

# Run sweep
wandb.agent(sweep_id, train, count=35)

Create sweep with ID: 1i5lqabw
Sweep URL: https://wandb.ai/da24m004-iitmaana/inaturalist-cnn-sweep_sample_02/sweeps/1i5lqabw


wandb: Agent Starting Run: u3n6k8ap with config:
wandb: 	activation: mish
wandb: 	batch_size: 128
wandb: 	dense_neurons: 128
wandb: 	dropout_rate: 0.2
wandb: 	epochs: 7
wandb: 	filter_size: 3
wandb: 	filter_strategy: double
wandb: 	learning_rate: 0.00872506802859034
wandb: 	num_filters: 64
wandb: 	seed: 42
wandb: 	use_augmentation: False
wandb: 	use_batch_norm: False
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch: 0, Batch: 0, Loss: 2.3072
Best model saved with val_acc: 10.00% at epoch 0
Epoch: 0, Train Loss: 501.0558, Train Acc: 10.13%, Val Loss: 2.3042, Val Acc: 10.00%
Epoch: 1, Batch: 0, Loss: 2.3107
Epoch: 1, Train Loss: 2.4239, Train Acc: 9.89%, Val Loss: 2.3028, Val Acc: 10.00%
Epoch: 2, Batch: 0, Loss: 2.3028
Epoch: 2, Train Loss: 2.4012, Train Acc: 9.11%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch: 3, Batch: 0, Loss: 2.3030
Epoch: 3, Train Loss: 2.3030, Train Acc: 9.86%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch: 4, Batch: 0, Loss: 2.3025
Epoch: 4, Train Loss: 2.3034, Train Acc: 9.21%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch: 5, Batch: 0, Loss: 2.3033
Epoch: 5, Train Loss: 2.3031, Train Acc: 9.56%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch: 6, Batch: 0, Loss: 2.3039
Epoch: 6, Train Loss: 2.3927, Train Acc: 9.55%, Val Loss: 2.3026, Val Acc: 10.00%


epoch,▁▂▃▅▆▇█
train_acc,█▆▁▆▂▄▄
train_loss,█▁▁▁▁▁▁
val_acc,▁▁▁▁▁▁▁
val_loss,█▂▁▁▁▁▁
epoch,6
train_acc,9.55119
train_loss,2.39273
val_acc,10
val_loss,2.30263


wandb: Agent Starting Run: fj9mpy23 with config:
wandb: 	activation: silu
wandb: 	batch_size: 128
wandb: 	dense_neurons: 128
wandb: 	dropout_rate: 0.4
wandb: 	epochs: 7
wandb: 	filter_size: 3
wandb: 	filter_strategy: same
wandb: 	learning_rate: 0.0051511926357889135
wandb: 	num_filters: 64
wandb: 	seed: 42
wandb: 	use_augmentation: False
wandb: 	use_batch_norm: False


...

Epoch: 0, Batch: 0, Loss: 2.3291
Best model saved with val_acc: 20.80% at epoch 0
Epoch: 0, Train Loss: 2.2849, Train Acc: 15.68%, Val Loss: 2.1177, Val Acc: 20.80%
Epoch: 1, Batch: 0, Loss: 2.1651
Best model saved with val_acc: 26.50% at epoch 1
Epoch: 1, Train Loss: 2.1421, Train Acc: 22.24%, Val Loss: 2.0375, Val Acc: 26.50%
Epoch: 2, Batch: 0, Loss: 2.1477
Best model saved with val_acc: 29.10% at epoch 2
Epoch: 2, Train Loss: 2.0989, Train Acc: 24.63%, Val Loss: 2.0074, Val Acc: 29.10%
Epoch: 3, Batch: 0, Loss: 2.1392
Best model saved with val_acc: 29.70% at epoch 3
Epoch: 3, Train Loss: 2.0720, Train Acc: 25.09%, Val Loss: 1.9959, Val Acc: 29.70%
Epoch: 4, Batch: 0, Loss: 2.0770
Epoch: 4, Train Loss: 2.0525, Train Acc: 25.45%, Val Loss: 1.9769, Val Acc: 28.45%
Epoch: 5, Batch: 0, Loss: 2.0473
Best model saved with val_acc: 31.90% at epoch 5
Epoch: 5, Train Loss: 2.0329, Train Acc: 26.90%, Val Loss: 1.9571, Val Acc: 31.90%
Epoch: 6, Batch: 0, Loss: 2.0264
Epoch: 6, Train Loss: 2.02

epoch,▁▂▃▅▆▇█
train_acc,▁▅▆▇▇██
train_loss,█▄▃▂▂▁▁
val_acc,▁▅▆▇▆██
val_loss,█▅▄▃▃▂▁
epoch,6
train_acc,27.62845
train_loss,2.02062
val_acc,31.6
val_loss,1.93309


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: n7p7ofzb with config:
wandb: 	activation: gelu
wandb: 	batch_size: 64
wandb: 	dense_neurons: 256
wandb: 	dropout_rate: 0.2
wandb: 	epochs: 7
wandb: 	filter_size: 3
wandb: 	filter_strategy: pyramid
wandb: 	learning_rate: 0.0007171894644397339
wandb: 	num_filters: 32
wandb: 	seed: 42
wandb: 	use_augmentation: False
wandb: 	use_batch_norm: True


Epoch: 0, Batch: 0, Loss: 2.4030
Epoch: 0, Batch: 100, Loss: 2.2033
Best model saved with val_acc: 27.90% at epoch 0
Epoch: 0, Train Loss: 2.1808, Train Acc: 20.18%, Val Loss: 2.0256, Val Acc: 27.90%
Epoch: 1, Batch: 0, Loss: 2.0797
Epoch: 1, Batch: 100, Loss: 2.0455
Best model saved with val_acc: 32.85% at epoch 1
Epoch: 1, Train Loss: 2.0538, Train Acc: 26.85%, Val Loss: 1.9547, Val Acc: 32.85%
Epoch: 2, Batch: 0, Loss: 2.1150
Epoch: 2, Batch: 100, Loss: 1.9760
Best model saved with val_acc: 32.90% at epoch 2
Epoch: 2, Train Loss: 1.9949, Train Acc: 29.42%, Val Loss: 1.9189, Val Acc: 32.90%
Epoch: 3, Batch: 0, Loss: 2.0282
Epoch: 3, Batch: 100, Loss: 1.9793
Best model saved with val_acc: 33.70% at epoch 3
Epoch: 3, Train Loss: 1.9603, Train Acc: 29.95%, Val Loss: 1.9149, Val Acc: 33.70%
Epoch: 4, Batch: 0, Loss: 2.0002
Epoch: 4, Batch: 100, Loss: 1.9148
Best model saved with val_acc: 34.00% at epoch 4
Epoch: 4, Train Loss: 1.9237, Train Acc: 31.63%, Val Loss: 1.8731, Val Acc: 34.00%


epoch,▁▂▃▅▆▇█
train_acc,▁▄▆▆▇▇█
train_loss,█▅▄▃▂▂▁
val_acc,▁▅▅▆▆▇█
val_loss,█▅▄▄▃▂▁
epoch,6
train_acc,34.30429
train_loss,1.86596
val_acc,36.7
val_loss,1.83095


In [14]:
def train_best_model(sweep_id, entity, project, epochs=100):
    """
    Load the best configuration from a W&B sweep and train a model for the specified epochs
    """
    # Get the best run from the sweep
    api = wandb.Api()
    sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
    
    # Find the best run based on validation accuracy
    best_run = None
    best_val_acc = 0
    
    for run in sweep.runs:
        if run.summary.get("val_acc", 0) > best_val_acc:
            best_val_acc = run.summary.get("val_acc", 0)
            best_run = run
    
    if best_run is None:
        print("No valid runs found in the sweep")
        return
    
    # Get the best configuration
    best_config = best_run.config
    print(f"Best run: {best_run.name}")
    print(f"Best validation accuracy: {best_val_acc:.2f}%")
    print(f"Best configuration: {best_config}")
    
    # Add epochs to the config
    best_config["epochs"] = epochs
    
    # Train a new model with the best configuration
    with wandb.init(project=wandb.run.project, config=best_config, name=f"best_model_100_epochs"):
        config = wandb.config
        
        # Set random seed for reproducibility
        torch.manual_seed(config.seed if "seed" in config else 42)
        np.random.seed(config.seed if "seed" in config else 42)
        
        # Data transforms
        if config.use_augmentation if "use_augmentation" in config else True:
            train_transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(10),
                transforms.ColorJitter(brightness=0.2, contrast=0.2),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
            ])
        else:
            train_transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
            ])

        val_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225])
        ])
        
        # Load dataset
        train_dataset = datasets.ImageFolder(
            root='/kaggle/working/inaturalist_12K/train',
            transform=train_transform
        )
        
        # Split into train and validation
        train_indices, val_indices = train_test_split(
            list(range(len(train_dataset))),
            test_size=0.2,
            stratify=train_dataset.targets,
            random_state=config.seed if "seed" in config else 42
        )
        
        train_subset = torch.utils.data.Subset(train_dataset, train_indices)
        val_subset = torch.utils.data.Subset(train_dataset, val_indices)
        
        train_loader = DataLoader(
            train_subset,
            batch_size=config.batch_size,
            shuffle=True,
            num_workers=4
        )
        
        val_loader = DataLoader(
            val_subset,
            batch_size=config.batch_size,
            shuffle=False,
            num_workers=4
        )
        
        # Initialize model
        model = FlexibleCNN(
            num_filters=config.num_filters,
            filter_size=config.filter_size,
            activation=config.activation if "activation" in config else "relu",
            dense_neurons=config.dense_neurons,
            use_batch_norm=config.use_batch_norm if "use_batch_norm" in config else True,
            dropout_rate=config.dropout_rate if "dropout_rate" in config else 0.2,
            filter_strategy=config.filter_strategy if "filter_strategy" in config else "same"
        )
        
        # Move model to GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)
        
        # Loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
        
        # Training loop
        best_val_acc = 0
        
        for epoch in range(config.epochs):
            # Training phase
            model.train()
            train_loss = 0
            train_correct = 0
            train_total = 0
            
            for batch_idx, (inputs, targets) in enumerate(train_loader):
                inputs, targets = inputs.to(device), targets.to(device)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item()
                _, predicted = outputs.max(1)
                train_total += targets.size(0)
                train_correct += predicted.eq(targets).sum().item()
                
                if batch_idx % 100 == 0:
                    print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}')
            
            train_acc = 100. * train_correct / train_total
            
            # Validation phase
            model.eval()
            val_loss = 0
            val_correct = 0
            val_total = 0
            
            with torch.no_grad():
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    
                    val_loss += loss.item()
                    _, predicted = outputs.max(1)
                    val_total += targets.size(0)
                    val_correct += predicted.eq(targets).sum().item()
            
            val_acc = 100. * val_correct / val_total
            
            # Log metrics to wandb
            wandb.log({
                "epoch": epoch,
                "train_loss": train_loss / len(train_loader),
                "train_acc": train_acc,
                "val_loss": val_loss / len(val_loader),
                "val_acc": val_acc
            })
            
            # Save best model
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(model.state_dict(), 'best_model_100_epochs.pth')
                print(f"Best model saved with val_acc: {val_acc:.2f}% at epoch {epoch}")
            
            print(f'Epoch: {epoch}, Train Loss: {train_loss/len(train_loader):.4f}, '
                  f'Train Acc: {train_acc:.2f}%, Val Loss: {val_loss/len(val_loader):.4f}, '
                  f'Val Acc: {val_acc:.2f}%')
        
        # Load test dataset and evaluate
        test_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225])
        ])
        
        test_dataset = datasets.ImageFolder(
            root='/kaggle/working/inaturalist_12K/val',
            transform=test_transform
        )
        
        test_loader = DataLoader(
            test_dataset,
            batch_size=config.batch_size,
            shuffle=False,
            num_workers=4
        )
        
        # Load best model weights
        model.load_state_dict(torch.load('best_model_100_epochs.pth'))
        model.eval()
        
        # Evaluate on test set
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        test_accuracy = 100 * correct / total
        print(f'Test Accuracy on {total} test images: {test_accuracy:.2f}%')
        wandb.log({"test_accuracy": test_accuracy})
        
        # Plot class predictions
        fig = plot_class_predictions(model, test_loader, test_dataset)
        wandb.log({"Class Predictions": wandb.Image(fig)})
        plt.close(fig)
        
        return model, test_accuracy

In [ ]:
# def train_best_model(sweep_id, entity, project, epochs=100)

In [15]:
import matplotlib.pyplot as plt
def plot_class_predictions(model, test_loader, test_dataset, num_classes=10, samples_per_class=3):
    """
    Plot a grid of images for each class with their predictions
    """
    # Get class names
    class_names = test_dataset.classes
    
    # Dictionary to store samples for each class
    class_samples = {i: [] for i in range(num_classes)}
    
    # Collect samples for each class
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.cpu().numpy()
            
            # Get model predictions
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)
            predictions = predictions.cpu().numpy()
            
            # Store samples for each class
            for i, (image, label, pred) in enumerate(zip(images, labels, predictions)):
                if len(class_samples[label]) < samples_per_class:
                    class_samples[label].append((image.cpu(), pred))
                
            # Check if we have enough samples for each class
            if all(len(samples) >= samples_per_class for samples in class_samples.values()):
                break
    
    # Create a grid of images
    fig, axes = plt.subplots(num_classes, samples_per_class, figsize=(12, 30))
    
    for class_idx in range(num_classes):
        for sample_idx in range(samples_per_class):
            if sample_idx < len(class_samples[class_idx]):
                img, pred = class_samples[class_idx][sample_idx]
                
                # Convert the image for display
                img = img.numpy()
                img = np.transpose(img, (1, 2, 0))  # CxHxW → HxWxC
                img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])  # Unnormalize
                img = np.clip(img, 0, 1)
                
                # Display the image
                axes[class_idx, sample_idx].imshow(img)
                axes[class_idx, sample_idx].set_title(f"True: {class_names[class_idx]}\nPred: {class_names[pred]}", 
                                                  fontsize=8,
                                                  color="green" if class_idx == pred else "red")
                axes[class_idx, sample_idx].axis('off')
            else:
                axes[class_idx, sample_idx].axis('off')
    
    plt.tight_layout()
    return fig

In [17]:
wandb.init(project="inaturalist-cnn-sweep_sample_02") 
model,_ = train_best_model(sweep_id, "da24m004-iitmaana", "inaturalist-cnn-sweep_sample_02", epochs=100)

Best run: nf_32_fs_3_strat_pyramid_act_gelu_dn_256_lr_0.0007171894644397339_bs_64_bn_True_dr_0.2_aug_False
Best validation accuracy: 36.70%
Best configuration: {'seed': 42, 'epochs': 7, 'activation': 'gelu', 'batch_size': 64, 'filter_size': 3, 'num_filters': 32, 'dropout_rate': 0.2, 'dense_neurons': 256, 'learning_rate': 0.0007171894644397339, 'use_batch_norm': True, 'filter_strategy': 'pyramid', 'use_augmentation': False}


Epoch: 0, Batch: 0, Loss: 2.4030
Epoch: 0, Batch: 100, Loss: 2.2099
Best model saved with val_acc: 27.85% at epoch 0
Epoch: 0, Train Loss: 2.1812, Train Acc: 20.14%, Val Loss: 2.0284, Val Acc: 27.85%
Epoch: 1, Batch: 0, Loss: 2.0783
Epoch: 1, Batch: 100, Loss: 2.0327
Best model saved with val_acc: 32.25% at epoch 1
Epoch: 1, Train Loss: 2.0536, Train Acc: 27.02%, Val Loss: 1.9499, Val Acc: 32.25%
Epoch: 2, Batch: 0, Loss: 2.1040
Epoch: 2, Batch: 100, Loss: 1.9826
Best model saved with val_acc: 32.90% at epoch 2
Epoch: 2, Train Loss: 1.9933, Train Acc: 29.30%, Val Loss: 1.9175, Val Acc: 32.90%
Epoch: 3, Batch: 0, Loss: 2.0119
Epoch: 3, Batch: 100, Loss: 1.9856
Best model saved with val_acc: 34.05% at epoch 3
Epoch: 3, Train Loss: 1.9593, Train Acc: 30.19%, Val Loss: 1.9164, Val Acc: 34.05%
Epoch: 4, Batch: 0, Loss: 1.9966
Epoch: 4, Batch: 100, Loss: 1.9228
Epoch: 4, Train Loss: 1.9253, Train Acc: 31.44%, Val Loss: 1.8782, Val Acc: 33.90%
Epoch: 5, Batch: 0, Loss: 1.7905
Epoch: 5, Batch:

/tmp/ipykernel_31/3446431227.py:203: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model_100_epochs.pth'))


Test Accuracy on 2000 test images: 35.35%


epoch,▁▂▃▅▆▇█
test_accuracy,▁
train_acc,▁▄▆▆▇▇█
train_loss,█▅▄▃▂▂▁
val_acc,▁▅▅▆▆▇█
val_loss,█▅▄▄▃▂▁
epoch,6
test_accuracy,35.35
train_acc,34.24178
train_loss,1.8678
val_acc,36.55


In [28]:
# Define transform
# test_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], 
#                          std=[0.229, 0.224, 0.225])
# ])

# # Load test dataset
# test_dataset = datasets.ImageFolder(
#     root='/kaggle/working/inaturalist_12K/val',
#     transform=test_transform
# )

# test_loader = DataLoader(
#     test_dataset,
#     batch_size=config['batch_size'],  # Make sure `config` exists
#     shuffle=True,
#     num_workers=4
# )

In [ ]:
# /kaggle/working/inaturalist_12K/train